In [ ]:
%pip install aiohttp

In [ ]:
import os
import aiohttp
import asyncio
import pandas as pd
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import time

In [3]:
async def get_total_count(api_key):
    """전체 데이터 수를 조회하는 함수"""    
    url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/tbLnOpendataRentV/1/1/2025"    
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                data = await response.json()        
                return data['tbLnOpendataRentV']['list_total_count']
    except Exception as e:
        print(f"데이터 수 조회 중 오류 발생: {e}")
        return 0

async def get_rent_data(start_idx, end_idx, api_key):
    """임대차 정보를 조회하는 함수"""    
    url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/tbLnOpendataRentV/{start_idx}/{end_idx}/2025"    
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                if response.status == 200:
                    data = await response.json()
                    return data['tbLnOpendataRentV']['row']
                else:
                    print(f"API 오류 발생: 상태 코드 {response.status}")
                    return []
    except Exception as e:
        print(f"데이터 조회 중 오류 발생: {e}")
        return []

In [4]:
async def main():
    # .env 파일에서 환경변수 로드
    load_dotenv()
    api_key = os.getenv('SEOUL_LANDMARK_API')    

    # 전체 데이터 수 조회
    total_count = int(await get_total_count(api_key) * 0.2)
    print(f"전체 데이터 수: {total_count}")

    if total_count == 0:    
        raise Exception("데이터를 가져올 수 없습니다.")

    # 데이터를 저장할 리스트
    all_data = []

    # 페이지 크기를 1000으로 설정
    page_size = 1000

    # 전체 반복 횟수 계산 (1000건 단위로)
    total_iterations = (total_count + page_size - 1) // page_size

    # 1000페이지씩 데이터 조회 (tqdm으로 진행률 표시)
    progress_bar = tqdm(range(1, total_count + 1, page_size), 
                    total=total_iterations,
                    desc="데이터 수집 진행률",
                    unit="천건")

    for start in progress_bar:
        end = min(start + page_size - 1, total_count)
        data = await get_rent_data(start, end, api_key)
        all_data.extend(data)
        
        # 진행상황 업데이트 메시지 (천 단위로 표시)
        progress_bar.set_postfix(현재=f"{start//1000}k~{(end//1000)}k")

    # 데이터프레임 생성
    df = pd.DataFrame(all_data)

    # CSV 파일로 저장
    current_time = time.strftime("%Y%m%d_%H%M%S")
    filename = f"seoul_rent_data_{current_time}.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"데이터가 {filename}에 저장되었습니다.")

    # 데이터 요약 정보 출력
    print("\n데이터 요약:")
    print(f"총 행 수: {len(df)}")
    print("\n컬럼 정보:")
    for col in df.columns:
        print(f"- {col}")

# 비동기 함수 실행
await main()

전체 데이터 수: 92065


데이터 수집 진행률:   0%|          | 0/93 [00:00<?, ?천건/s]

NameError: name 'time' is not defined